<a href="https://colab.research.google.com/github/v1umahmo/HonoursProject/blob/main/HonoursCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Necessary Libraries

In [ ]:
#For data processing
import pandas as pd
#For linear algebra
import numpy as np
#Regular expression library
import re
import nltk

### Import the Training and Testing Data

In [ ]:
train = pd.read_csv("") # Import training data from file path
test = pd.read_csv("") # Import test data from file path

### Look At The Training Data

In [ ]:
train.info() # Print training data info

In [ ]:
train.head(10) # Print the first ten tweets from the training data

In [ ]:
train.tail(10) # Print the last ten tweets from the training data

### Clean The Training Data

In [ ]:
# For loop to go through each tweet for the data cleaning
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, ' ', input_txt)
  return input_txt

In [ ]:
# Get Rid of the user handles
train['cleaned_tweets'] = np.vectorise(remove_pattern)(train['tweet'], "@[\w]*")

In [ ]:
train.head(10) # Print the first ten tweets from the training data with their twitter handles removed

In [ ]:
# Make tweets lower case
train['cleaned_tweets'] = train['cleaned_tweets'].apply(lambda x: x.lower())

In [ ]:
train.head(10) # Print the first ten tweets from the training data with the tweets lower case

In [ ]:
# Remove hashtags and special characters
train['cleaned_tweets'] = train['cleaned_tweets'].apply(lambda x: re.sub(r'[^\w\s]',' ',x))

In [ ]:
train.head(10)

In [ ]:
# Remove numbers etc.
train['cleaned_tweets'] = train['cleaned_tweets'].apply(lambda x: re.sub(r'[^a-zA-z]',' ',x))

In [ ]:
train.head(10)

In [ ]:
# Import libraries for tokenization and stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Create tokenization for the cleaned tweets
train['tokenized_tweets'] = train['cleaned_tweets'].apply (lambda x: word_tokenize(x))

In [ ]:
train.head(10)

In [ ]:
# Use this to remove words such as if, but, can etc. (Words that have no value)
stop_words = set(stopwords.words('english'))
stop_words

In [ ]:
# Apply stopwords to the tokenized tweets
train['tokenized_tweets_filtered'] = train['tokenized_tweets'].apply(lambda x: [word for word in x if not word in stop_words])

In [ ]:
train.head(10)

In [ ]:
#Import library for stemming
from nltk.stem import PorterStemmer
stemming = PorterStemmer()

In [ ]:
#Stem the tweets and create another colim for tweets that are stemmed
train['stemmed_tweets'] = train['tokenized_tweets_filtered'].apply(lambda x: ' '.join([stemming.step(i) for i in x]))

In [ ]:
train.head(10)

In [ ]:
#Import library for lemmatizing
from nltk.stem.wordnet import WordnetLemmatizer
lemmatizing = WordnetLemmatizer()

In [ ]:
# Lemmatize the tweets and create another column for tweets that are lemmatized
train['lemmatized_tweets'] = train['tokenized_tweets_filtered'].apply(lambda x: ' ' .join([lemmatizing.lemmatize(i) for i in x]))

In [ ]:
train.head(10)

### Apply Feature Extraction

In [ ]:
# Import library
from sklearn.feature_extraction.text import CountVectorizer

### Clean The Test Data

In [ ]:
test.info()

In [ ]:
test.head(10)

In [ ]:
test.tail(10)

In [ ]:
# Get Rid of the user handles
test['cleaned_tweets'] = np.vectorise(remove_pattern)(test['tweet'], "@[\w]*")

In [ ]:
test.head(10)

In [ ]:
# Make tweets lower case
test['cleaned_tweets'] = test['cleaned_tweets'].apply(lambda x: x.lower())

In [ ]:
test.head(10)

In [ ]:
test['cleaned_tweets'] = test['cleaned_tweets'].apply(lambda x: re.sub(r'[^\w\s]',' ',x))

In [ ]:
test.head(10)

In [ ]:
test['cleaned_tweets'] = test['cleaned_tweets'].apply(lambda x: re.sub(r'[^a-zA-z]',' ',x))

In [ ]:
test.head(10)

In [ ]:
# Import libraries for tokenization and stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Create tokenization for the cleaned tweets
test['tokenized_tweets'] = test['cleaned_tweets'].apply (lambda x: word_tokenize(x))

In [ ]:
test.head(10)

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words

In [ ]:
test['tokenized_tweets_filtered'] = test['tokenized_tweets'].apply(lambda x: [word for word in x if not word in stop_words])

In [ ]:
test.head(10)

In [ ]:
#Import library for stemming
from nltk.stem import PorterStemmer
stemming = PorterStemmer()

In [ ]:
#Stem the tweets and create another colim for tweets that are stemmed
test['stemmed_tweets'] = test['tokenized_tweets_filtered'].apply(lambda x: ' '.join([stemming.step(i) for i in x]))

In [ ]:
test.head(10)

In [ ]:
#Import library for lemmatizing
from nltk.stem.wordnet import WordnetLemmatizer
lemmatizing = WordnetLemmatizer()

In [ ]:
test['lemmatized_tweets'] = test['tokenized_tweets_filtered'].apply(lambda x: ' ' .join([lemmatizing.lemmatize(i) for i in x]))

In [ ]:
test.head(10)